In [2]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from empyrical import max_drawdown, alpha, beta
from sklearn.preprocessing import MinMaxScaler
from src.twitter_interface import TwitterInterface
from src.asset_selector import AssetSelector
from src.indicators import Indicators
from util import time_formatter
from src.predictor import Predictor
from sklearn.cluster import KMeans
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as py
import pandas as pd
import numpy as np
import configparser
import requests
import twitter
import json
import time
import sys
import os

weeks = 13

backdate = time_formatter(time.time() - (604800 * weeks))
config = configparser.ConfigParser()
try:
    config.read(os.path.relpath("config.ini"))
except FileExistsError as e:
    print("FileExistsError: {}".format(e))
    sys.exit(1)
alpaca_api = tradeapi.REST(
    base_url    = config["alpaca"]["APCA_API_BASE_URL"],
    key_id      = config["alpaca"]["APCA_API_KEY_ID"],
    secret_key  = config["alpaca"]["APCA_API_SECRET_KEY"],
    api_version = config["alpaca"]["VERSION"]
)
twitter_api = twitter.Api(
    config["twitter"]["CONSUMER_KEY"],
    config["twitter"]["CONSUMER_SECRET"],
    config["twitter"]["ACCESS_TOKEN_KEY"],
    config["twitter"]["ACCESS_TOKEN_SECRET"]
)
tq_key = config["tenquant"]["api_key"]

tdf = pd.DataFrame()
ti = TwitterInterface(twitter_api, tdf)
trading_account = alpaca_api.get_account()

selector = AssetSelector(alpaca_api, edgar_token=None)
indicators = Indicators(alpaca_api)

def get_sentiment(text):
    """Given a text block, return a sentiment score based.
    :param text:
    :return:
    """
    text_polarity   = sid.polarity_scores(text)

    if text_polarity["compound"] > 0.50:
        sentiment   = "positive"
    else:
        sentiment   = "negative"
    return sentiment, text_polarity["compound"]

def get_losers():
    print("Losers".center(45))
    print()
    print("Ticker".ljust(10), "Last".ljust(11), "Change".ljust(11), "Pct Loss")
    print("{:<30}".format("–" * 45))
    losers = alpaca_api.polygon.gainers_losers("losers")
    
    for symbol in range(len(losers)):
        
        # print out some indicators here # TODO
        # also throw out shitty stocks
        
        print(alpaca_api.polygon.gainers_losers("losers")[symbol].ticker.ljust(10),
              "$" + str(alpaca_api.polygon.gainers_losers("losers")[symbol].lastTrade["p"]).ljust(10),
              "$" + str(alpaca_api.polygon.gainers_losers("losers")[symbol].todaysChange).ljust(10),
              str(alpaca_api.polygon.gainers_losers("losers")[symbol].todaysChangePerc)+"%")

def get_gainers():
    print("Gainers".center(45))
    print()
    print("Ticker".ljust(10), "Last".ljust(11), "Change".ljust(11), "Pct Gain".ljust(11), "Buy?")
    print("{:<30}".format("–" * 45))
    
    gainers = alpaca_api.polygon.gainers_losers()
    
    for symbol in range(len(gainers)):
        
        ticker = gainers[symbol].ticker
        bars = alpaca_api.get_barset(ticker, "1D", after=backdate)
        dataframe = selector.extract_bar_data(bars, ticker)
        
        
        _macd = indicators.get_macd(dataframe)
        macd = _macd["MACD"]
        signal = _macd["SIGNAL"]
        
        # I think this might be my final MACD buy signal
        
        try:
            buysignal = macd.iloc[-1] < 0 and min(macd.iloc[-4:-2]) < signal.iloc[-1] and macd.iloc[-1] > signal.iloc[-1]
        except IndexError:
            # print("Throwing {} away due to index errors. Not worth the headache -- will handle later".format(ticker))
            continue
        
        # print(macd.iloc[-1], signal.iloc[-1], )
        # print("Buy? ", buysignal)
        
        print(alpaca_api.polygon.gainers_losers()[symbol].ticker.ljust(10),
            "$" + str(alpaca_api.polygon.gainers_losers()[symbol].lastTrade["p"]).ljust(10),
            "$" + str(alpaca_api.polygon.gainers_losers()[symbol].todaysChange).ljust(10),
            str(alpaca_api.polygon.gainers_losers()[symbol].todaysChangePerc)+"%".ljust(10), 
            str(buysignal))

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\benjamin.beale\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
if __name__ == "__main__":
    get_gainers()
    print()
    get_losers()
    print()
    print("Bulls".center(45))
    bulls = selector.bullish_candlesticks(64, 20)
    print()
    print("Bears".center(45))
    bears = selector.bearish_candlesticks(64, 20)

                   Gainers                   

Ticker     Last        Change      Pct Gain    Buy?
–––––––––––––––––––––––––––––––––––––––––––––
ZVZZT      $980        $921.99     1589.364%          False
ALIM       $4.6        $4.295      1408.197%          False
ESTRW      $0.01       $0.006      177.778%          False
AIHS       $1.06       $0.635      149.412%          False
TKKS       $10.225     $4.325      73.305%          True
PSV        $10.225     $0.415      65.847%          False
KERNW      $1.045      $0.25       65.847%          True
TKKSU      $0.64       $0.25       46.309%          True
ANY        $10.9       $3.45       46.309%          False
ANY        $1.09       $0.334      44.218%          False
DSKEW      $0.22       $0.009      37.5%          False
FTCH       $9.67       $0.009      30%          True
MLNT       $9.67       $0.39       29.278%          False
MTC        $2.2957     $0.496      27.539%          False
CAAS       $2.6        $0.496      27.539%     

In [8]:
ticker = "FTCH"

In [9]:
sid = SentimentIntensityAnalyzer()
stocktwits = requests.post("https://api.stocktwits.com/api/2/streams/symbol/{}.json".format(ticker))
stocktwits = json.loads(stocktwits.text)
stocktwitstext = "\n".join([m["body"] for m in stocktwits["messages"]])
tweets = ti.get_tweets(ticker, "stock")

In [10]:
stsentiment, stscore = get_sentiment(stocktwitstext)
tsentiment, tscore = get_sentiment(tweets)

In [11]:
print("Sentiment score for {}".format(ticker))
print("{:<30}".format("–" * 45))
print("Stocktwits:\t{}\t score: {}\nTwitter:\t{}\t score:\t{}".format(stsentiment, stscore, tsentiment, tscore))
print("Average:\t{}".format((stscore + tscore)/2))

Sentiment score for FTCH
–––––––––––––––––––––––––––––––––––––––––––––
Stocktwits:	positive	 score: 0.9748
Twitter:	positive	 score:	0.9851
Average:	0.97995


In [12]:
# alpaca_api.polygon.historic_trades(ticker, "2019-10-10", limit=10)
# alpaca_api.polygon.news(ticker)
# alpaca_api.polygon.last_trade(ticker)
# alpaca_api.polygon.last_quote(ticker)
# alpaca_api.polygon.company(ticker)
# alpaca_api.polygon.dividends(ticker)
# alpaca_api.polygon.financials(ticker)

In [13]:
tenquant_data = requests.get("https://api.tenquant.io/data?key={}&ticker={}".format(tq_key, ticker))
tenquant_data = json.loads(tenquant_data.text)

In [14]:
stocktwits

{'response': {'status': 200},
 'symbol': {'id': 14844,
  'symbol': 'FTCH',
  'title': 'Farfetch Ltd',
  'aliases': [],
  'is_following': False,
  'watchlist_count': 1604},
 'cursor': {'more': True, 'since': 183975899, 'max': 183936061},
 'messages': [{'id': 183975899,
   'body': 'Farfetch&#39;s PT cut by KeyCorp to $20.00. https://www.marketbeat.com/r/1304382 $FTCH',
   'created_at': '2019-11-16T01:00:40Z',
   'user': {'id': 284225,
    'username': 'AnalystRatingsNetwork',
    'name': 'MarketBeat.com',
    'avatar_url': 'https://avatars.stocktwits.com/production/284225/thumb-1484325936.png',
    'avatar_url_ssl': 'https://avatars.stocktwits.com/production/284225/thumb-1484325936.png',
    'join_date': '2013-11-24',
    'official': False,
    'identity': 'User',
    'classification': [],
    'followers': 7748,
    'following': 9,
    'ideas': 762121,
    'watchlist_stocks_count': 0,
    'like_count': 9,
    'plus_tier': '',
    'premium_room': ''},
   'source': {'id': 1022, 'title': 'Za

In [15]:
tweets 

'Stock Traders Purchase Large Volume of Put Options on Farfetch $FTCH https://t.co/isRQVNqBMD\n\nStock Traders Purchase Large Volume of Put Options on Farfetch $FTCH  https://t.co/ROfEZDtZ5Z\n\nFarfetch $FTCH Stock Price Up 29.3% Following Earnings Beat https://t.co/sLOBFzj79N\n\n$SPY $QQQ $IWM $DIA $UVXY Top stock stories for today $CI $CNC $CVS $HUM $UNH $WCG $HNT $MOH $ANTM $HCA $CYH $LPNT… https://t.co/KOhqUNsHxE\n\n$FTCH:\n\nFarfetch Earnings: FTCH Stock Rockets 29% on Q3 Results:\n\nhttps://t.co/etcL4wldkS\n\n$FTCH was soaring today on a strong Q3 earnings beat. https://t.co/x7QMc8jG7d\n\n$REAL closing red when the following happened today:\n\n- $SPY $SPX Rallying heavily\n- Peer in segment $FTCH reportin… https://t.co/Bj6IeDD1FT\n\n$FTCH rallies, gaining $2.19 (29.28%) within a single day on high volume: https://t.co/ebfg6YNxUr\nSentiment: Slight… https://t.co/BHqTXL17yp\n\n$FTCH \nMaintainsKeyBancOverweight27 » 20\nMaintainsUBSBuy18 » 15\nMaintainsCredit SuisseOutperform22 » 17

In [16]:
tenquant_data

{'assets': 1351393000.0,
 'bookvalue': 1128431000.0,
 'comprehensiveincome': -179281000.0,
 'comprehensiveincomeattributabletononcontrollinginterest': 0.0,
 'comprehensiveincomeattributabletoparent': -179281000.0,
 'costofrevenue': 0,
 'currencycode': 'USD',
 'currentassets': 1199410000.0,
 'currentliabilities': 194158000.0,
 'date': '2018-12-31',
 'dividendpayments': 0.0,
 'dividendyield': 0.0,
 'documenttype': '20-F',
 'duration': 4,
 'equity': 1128431000.0,
 'exchangegainslosses': 0.0,
 'extraordaryitemsgainloss': 0.0,
 'grossprofit': 298450000.0,
 'incomebeforeequitymethodinvestments': -155575000.0,
 'incomefromcontinuingoperationsaftertax': -155575000.0,
 'incomefromcontinuingoperationsbeforetax': -155575000.0,
 'incomefromequitymethodinvestments': 0.0,
 'incometaxexpensebenefit': 0.0,
 'interestanddebtexpense': 0.0,
 'liabilities': 222962000.0,
 'liabilitiesandequity': 1351393000.0,
 'marketcap': 1922352520,
 'netcashflow': 679783000.0,
 'netcashflowsfinancing': 859526000.0,
 'ne